In [1]:
import torch
from torch.utils.data import DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import torch.nn as nn  
import torch.optim as optim

class CustomDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        self.features = self.data.iloc[:, :-1].values
        self.targets = self.data.iloc[:, -1].values

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = torch.tensor(self.features[idx], dtype=torch.float32)
        target = torch.tensor(self.targets[idx], dtype=torch.float32)
        if torch.cuda.is_available():
            # CUDA is available, print CUDA device information
            print("CUDA is available! Using GPU.")
            print("Number of CUDA devices:", torch.cuda.device_count())
            print("CUDA device name:", torch.cuda.get_device_name(0))  # Assuming you have one GPU, get its name
            # Move tensors to GPU
            features = self.features[idx].to('cuda')
            target = self.targets[idx].to('cuda')
            print("Tensor device:", features.device)
        return features, target

csv_file = "./MEFAR_MID.csv"
dataset = CustomDataset(csv_file)

train_set, test_set = train_test_split(dataset, test_size=0.2, random_state=42)

train_loader = DataLoader(train_set, batch_size=10, shuffle=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False)

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.in_dim = 17
        self.out_dim = 1
        self.fc1 = nn.Linear(self.in_dim, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 4)
        self.fc4 = nn.Linear(4, 2)
        self.fc5 = nn.Linear(2, self.out_dim)
        self.relu = nn.ReLU()
        self.log_softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x):
        a1 = self.relu(self.fc1(x))
        a2 = self.relu(self.fc2(a1))
        a3 = self.relu(self.fc3(a2))
        a4 = self.relu(self.fc4(a3))
        output = self.fc5(a4)  # Pas besoin de log_softmax pour la régression
        return output

model = MLP()
criterion = nn.MSELoss()  # Utilisation de MSELoss pour la régression
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(2):
    running_loss = 0.0
    ancienRunning=-1
    for i, data in enumerate(train_loader, 0):
        if (abs(ancienRunning-running_loss)<0.001):
            optimizer = optim.SGD(model.parameters(), lr=0.000001)
            
        ancienRunning=running_loss
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)  # Squeeze pour éliminer les dimensions inutiles
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        
        if (i + 1) % 2000 == 0:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')


CUDA is available! Using GPU.
Number of CUDA devices: 4
CUDA device name: NVIDIA A10G


AttributeError: 'numpy.ndarray' object has no attribute 'to'

[1,  2000] loss: 0.253
[1,  4000] loss: 0.253
[1,  6000] loss: 0.253
[1,  8000] loss: 0.253
[1, 10000] loss: 0.253
[1, 12000] loss: 0.253
[1, 14000] loss: 0.253
[1, 16000] loss: 0.253
[1, 18000] loss: 0.252
[1, 20000] loss: 0.253
[1, 22000] loss: 0.253
[1, 24000] loss: 0.252
[1, 26000] loss: 0.253
[1, 28000] loss: 0.253
[1, 30000] loss: 0.252
[1, 32000] loss: 0.253
[1, 34000] loss: 0.253
[1, 36000] loss: 0.253
[1, 38000] loss: 0.253
[1, 40000] loss: 0.253
[1, 42000] loss: 0.253
[1, 44000] loss: 0.252
[1, 46000] loss: 0.253
[1, 48000] loss: 0.253
[1, 50000] loss: 0.253
[1, 52000] loss: 0.253
[1, 54000] loss: 0.253
[1, 56000] loss: 0.253
[1, 58000] loss: 0.253
[1, 60000] loss: 0.253
[1, 62000] loss: 0.253
[1, 64000] loss: 0.253
[1, 66000] loss: 0.253
[1, 68000] loss: 0.252
[1, 70000] loss: 0.253
[1, 72000] loss: 0.253
[2,  2000] loss: 0.253
[2,  4000] loss: 0.253
[2,  6000] loss: 0.253
[2,  8000] loss: 0.253
[2, 10000] loss: 0.253
[2, 12000] loss: 0.253
[2, 14000] loss: 0.252
[2, 16000] 